### IMPORTING LIBRARIES

In [62]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

### WEB SCRAPPING AND CLEANING DATA

In [4]:
#Getting data from url to pandas dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df=dfs[0]
df2=df[['Postal Code','Borough','Neighborhood']]
df2

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
#cleaning the data by removing not assigned values
df_filtered = df2[df2['Borough'] != 'Not assigned'] 
df_filtered

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### ASSIGNING LATITUDE AND LONGITUDES


In [7]:
#get the csv file with latitude and longitude data
df_coord=pd.read_csv('Geospatial_Coordinates.csv')
#concat with our data frames.
leftMerge=pd.merge(df_filtered, df_coord, on='Postal Code',how='left')
leftMerge

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(leftMerge['Borough'].unique()),
        leftMerge.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### GETTING VENUES OF ALL NEIGHBOURHOODS ACROSS ALL BOROUGH FROM FOURSQUARE

In [13]:
#foursquare credentials
CLIENT_ID = 'NSN1CSPWLJRDETZ1W2LPWRG2QKPUUSCG4X3XNLWEVEVNJBSR' # your Foursquare ID
CLIENT_SECRET = 'UORXJ2VXBY1YMCTM3VHRLV2OQLZANVLYPIHSEMAUGFEHYEZT' # your Foursquare Secret
VERSION = '20180604'
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NSN1CSPWLJRDETZ1W2LPWRG2QKPUUSCG4X3XNLWEVEVNJBSR
CLIENT_SECRET:UORXJ2VXBY1YMCTM3VHRLV2OQLZANVLYPIHSEMAUGFEHYEZT


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:

all_venues = getNearbyVenues(names=leftMerge['Neighborhood'],
                                   latitudes=leftMerge['Latitude'],
                                   longitudes=leftMerge['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [24]:
print(all_venues.shape)
all_venues.head()

(2142, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [25]:
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
...,...,...,...,...,...,...
"Willowdale, Willowdale West",7,7,7,7,7,7
Woburn,4,4,4,4,4,4
Woodbine Heights,7,7,7,7,7,7


In [26]:
print('There are {} uniques categories.'.format(len(all_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [33]:
# one hot encoding
all_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
all_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [all_onehot.columns[-1]] + list(all_onehot.columns[:-1])
all_onehot = all_onehot[fixed_columns]

all_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
all_grouped = all_onehot.groupby('Neighborhood').mean().reset_index()
all_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0
95,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [115]:
#getting top 10venues of each neighborhood
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_grouped['Neighborhood']

for ind in np.arange(all_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_grouped.iloc[ind, :], num_top_venues)



In [116]:
neighborhoods_venues_sorted.shape

(97, 31)

#### Following are the top 10 venues in all neighborhoods of toronto city

In [117]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,...,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Coworking Space,Creperie
1,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Sandwich Place,Skating Rink,Athletics & Sports,Pool,Pub,Coffee Shop,...,Department Store,Deli / Bodega,Cycle Studio,Comic Shop,Curling Ice,Cupcake Shop,Cuban Restaurant,College Stadium,Colombian Restaurant,Creperie
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Deli / Bodega,Sushi Restaurant,Middle Eastern Restaurant,Restaurant,Fried Chicken Joint,...,Dessert Shop,Department Store,Dog Run,Dance Studio,Women's Store,Cuban Restaurant,Cycle Studio,Curling Ice,Cupcake Shop,Donut Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Event Space,Cycle Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Juice Bar,Pharmacy,Pizza Place,Cupcake Shop,...,Greek Restaurant,Cycle Studio,Discount Store,Garden,Diner,Dim Sum Restaurant,Dessert Shop,Garden Center,Gas Station,Department Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Willowdale West",Grocery Store,Pharmacy,Bank,Home Service,Pizza Place,Butcher,Coffee Shop,Drugstore,Donut Shop,...,Cycle Studio,Dance Studio,Ethiopian Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center
93,Woburn,Coffee Shop,Indian Restaurant,Korean Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,...,Cycle Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping
94,Woodbine Heights,Park,Pharmacy,Beer Store,Skating Rink,Cosmetics Shop,Curling Ice,Video Store,Comfort Food Restaurant,Eastern European Restaurant,...,Department Store,Deli / Bodega,Dance Studio,Cycle Studio,College Stadium,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Colombian Restaurant
95,York Mills West,Park,Bank,Convenience Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Cycle Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Construction & Landscaping,Concert Hall


In [118]:
# set number of clusters
kclusters = 5

all_grouped_clustering = all_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [119]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = leftMerge

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Discount Store,Deli / Bodega,...,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,French Restaurant,Hockey Arena,Coffee Shop,Intersection,...,Cycle Studio,Ethiopian Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Bakery,Park,Pub,...,Yoga Studio,Mexican Restaurant,Performing Arts Venue,Cosmetics Shop,Gym / Fitness Center,Art Gallery,Health Food Store,Historic Site,Antique Shop,Deli / Bodega
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Clothing Store,Furniture / Home Store,Women's Store,Sporting Goods Shop,...,Dumpling Restaurant,Department Store,Cycle Studio,Dance Studio,Electronics Store,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Coffee Shop,Gym,Diner,Restaurant,...,Sandwich Place,Yoga Studio,Theater,Vegetarian / Vegan Restaurant,Sushi Restaurant,Coworking Space,Discount Store,Theme Restaurant,Dim Sum Restaurant,Dessert Shop


In [120]:
manhattan_merged=manhattan_merged.dropna()
convert_dict = {'Cluster Labels': int, 
               } 
  
manhattan_merged = manhattan_merged.astype(convert_dict) 
print(manhattan_merged.dtypes)

Postal Code                object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
11th Most Common Venue     object
12th Most Common Venue     object
13th Most Common Venue     object
14th Most Common Venue     object
15th Most Common Venue     object
16th Most Common Venue     object
17th Most Common Venue     object
18th Most Common Venue     object
19th Most Common Venue     object
20th Most Common Venue     object
21th Most Common Venue     object
22th Most Common Venue     object
23th Most Common Venue     object
24th Most Comm

In [121]:
manhattan_merged['Cluster Labels'].describe()

count    101.000000
mean       2.811881
std        0.560587
min        0.000000
25%        3.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: Cluster Labels, dtype: float64

In [122]:
latitude= 43.6532 
longitude=-79.3832

In [123]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)

ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [124]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
12,Scarborough,0,Bar,Moving Target,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall
94,Etobicoke,0,Rental Car Location,Drugstore,Bar,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,...,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convention Center,Cycle Studio,Cosmetics Shop,Coworking Space


In [125]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
6,Scarborough,1,Fast Food Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,...,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop


In [126]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,North York,2,Park,Food & Drink Shop,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,...,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant
21,York,2,Park,Pool,Women's Store,College Stadium,Colombian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,...,Cycle Studio,Ethiopian Restaurant,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,College Rec Center,Convenience Store
35,East York,2,Park,Coffee Shop,Intersection,Convenience Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,...,Ethiopian Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Construction & Landscaping,Concert Hall
49,North York,2,Park,Bakery,Construction & Landscaping,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,...,Cycle Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Concert Hall
61,Central Toronto,2,Park,Bus Line,Swim School,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,...,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Curling Ice,Convention Center,Cosmetics Shop
64,York,2,Park,Convenience Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant
66,North York,2,Park,Bank,Convenience Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,...,Cycle Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Construction & Landscaping,Concert Hall
68,Central Toronto,2,Park,Jewelry Store,Bus Line,Trail,Sushi Restaurant,Drugstore,Dumpling Restaurant,Donut Shop,...,Convenience Store,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convention Center,Cycle Studio,Cosmetics Shop
77,Etobicoke,2,Park,Bus Line,Pizza Place,Sandwich Place,Diner,Deli / Bodega,Department Store,Dessert Shop,...,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop
85,Scarborough,2,Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,...,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop


In [127]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
1,North York,3,French Restaurant,Hockey Arena,Coffee Shop,Intersection,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,...,Cycle Studio,Ethiopian Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store
2,Downtown Toronto,3,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Theater,Café,Distribution Center,...,Yoga Studio,Mexican Restaurant,Performing Arts Venue,Cosmetics Shop,Gym / Fitness Center,Art Gallery,Health Food Store,Historic Site,Antique Shop,Deli / Bodega
3,North York,3,Clothing Store,Furniture / Home Store,Women's Store,Sporting Goods Shop,Accessories Store,Event Space,Vietnamese Restaurant,Boutique,...,Dumpling Restaurant,Department Store,Cycle Studio,Dance Studio,Electronics Store,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space
4,Downtown Toronto,3,Coffee Shop,Gym,Diner,Restaurant,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,...,Sandwich Place,Yoga Studio,Theater,Vegetarian / Vegan Restaurant,Sushi Restaurant,Coworking Space,Discount Store,Theme Restaurant,Dim Sum Restaurant,Dessert Shop
7,North York,3,Gym,Asian Restaurant,Japanese Restaurant,Restaurant,Beer Store,Coffee Shop,Caribbean Restaurant,Athletics & Sports,...,Gym / Fitness Center,Grocery Store,Sandwich Place,Diner,Donut Shop,Drugstore,Dog Run,Dumpling Restaurant,Distribution Center,Eastern European Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,3,Coffee Shop,Café,Gym,Japanese Restaurant,Hotel,Restaurant,American Restaurant,Deli / Bodega,...,Theater,Ice Cream Shop,Sandwich Place,Salon / Barbershop,Bookstore,Vegetarian / Vegan Restaurant,Breakfast Spot,Pub,Plaza,Building
99,Downtown Toronto,3,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Hotel,Gastropub,...,Ramen Restaurant,Wings Joint,Grocery Store,Martial Arts Dojo,Fast Food Restaurant,Food & Drink Shop,Flower Shop,Afghan Restaurant,American Restaurant,Arts & Crafts Store
100,East Toronto,3,Light Rail Station,Pizza Place,Brewery,Skate Park,Burrito Place,Farmers Market,Fast Food Restaurant,Spa,...,Comfort Food Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Concert Hall,Diner,Dessert Shop,Construction & Landscaping,Convenience Store
101,Etobicoke,3,Baseball Field,Locksmith,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall


In [128]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
45,North York,4,Cafeteria,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall
